# Network-Specific ISO 8583 Implementations

While ISO 8583 is a standard, different payment networks have their own variations. This notebook covers VISA, Mastercard, and other network-specific implementations.

## Setup

In [ ]:
import sys

sys.path.insert(0, "..")

from iso8583sim.core.builder import ISO8583Builder
from iso8583sim.core.parser import ISO8583Parser
from iso8583sim.core.types import NETWORK_SPECIFIC_FIELDS, CardNetwork, ISO8583Message

## Network Types

iso8583sim supports several payment networks:

In [ ]:
print("Supported Networks:")
for network in CardNetwork:
    print(f"  - {network.name}: {network.value}")

## VISA Implementation

VISA uses the BASE I and BASE II systems with specific field definitions.

### VISA-Specific Fields

In [ ]:
# Check VISA-specific field definitions
visa_fields = NETWORK_SPECIFIC_FIELDS.get(CardNetwork.VISA, {})
print("VISA-Specific Fields:")
print("-" * 60)
for field_num, field_def in sorted(visa_fields.items()):
    print(f"Field {field_num:3d}: {field_def.description}")

### Building a VISA Authorization

In [ ]:
builder = ISO8583Builder()
parser = ISO8583Parser()

# VISA Authorization Request
visa_auth = ISO8583Message(
    mti="0100",
    network=CardNetwork.VISA,
    fields={
        0: "0100",
        2: "4111111111111111",  # Visa test card
        3: "000000",  # Purchase
        4: "000000010000",  # $100.00
        7: "1215143022",  # Transmission date/time
        11: "123456",  # STAN
        12: "143022",  # Local time
        13: "1215",  # Local date
        14: "2612",  # Expiry
        18: "5411",  # MCC: Grocery stores
        22: "051",  # POS entry mode (chip)
        23: "001",  # Card sequence number
        25: "00",  # POS condition code
        26: "12",  # PIN capture code
        32: "123456",  # Acquiring institution
        35: "4111111111111111=26125010000000000000",
        37: "123456789012",  # RRN
        41: "TERM0001",
        42: "MERCHANT123456 ",
        43: "ACME Grocery            San Francisco CA US",
        49: "840",  # USD
        52: "1234567890ABCDEF",  # PIN block
        # VISA-specific fields
        60: "0001",  # Additional POS info
        63: "VISA",  # Reserved private
    },
)

raw_visa = builder.build(visa_auth)
print("VISA Authorization Request:")
print(f"Length: {len(raw_visa)} bytes")
print(f"Raw: {raw_visa[:80]}...")

In [ ]:
# Parse with VISA context
parsed_visa = parser.parse(raw_visa, network=CardNetwork.VISA)

print("Parsed VISA Message:")
print(f"Network: {parsed_visa.network}")
print(f"MTI: {parsed_visa.mti}")
print("\nKey Fields:")
print(f"  PAN (F2): {parsed_visa.fields.get(2)}")
print(f"  MCC (F18): {parsed_visa.fields.get(18)}")
print(f"  Additional POS (F60): {parsed_visa.fields.get(60)}")

## Mastercard Implementation

Mastercard uses the GCMS (Global Card Management System) with its own variations.

### Mastercard-Specific Fields

In [ ]:
# Check Mastercard-specific field definitions
mc_fields = NETWORK_SPECIFIC_FIELDS.get(CardNetwork.MASTERCARD, {})
print("Mastercard-Specific Fields:")
print("-" * 60)
for field_num, field_def in sorted(mc_fields.items()):
    print(f"Field {field_num:3d}: {field_def.description}")

### Building a Mastercard Authorization

In [ ]:
# Mastercard Authorization Request
mc_auth = ISO8583Message(
    mti="0100",
    network=CardNetwork.MASTERCARD,
    fields={
        0: "0100",
        2: "5555555555554444",  # Mastercard test card
        3: "000000",
        4: "000000007500",  # $75.00
        7: "1215150000",
        11: "654321",
        12: "150000",
        13: "1215",
        14: "2712",
        18: "5812",  # MCC: Restaurants
        22: "051",
        23: "001",
        25: "00",
        32: "654321",
        35: "5555555555554444=27125010000000000000",
        37: "987654321098",
        41: "TERM0002",
        42: "RESTAURANT54321",
        43: "Fine Dining Restaurant   Los Angeles   CA US",
        49: "840",
        # Mastercard-specific fields
        48: "TCC:R",  # Additional data - Transaction Category Code
    },
)

raw_mc = builder.build(mc_auth)
print("Mastercard Authorization Request:")
print(f"Length: {len(raw_mc)} bytes")
print(f"Raw: {raw_mc[:80]}...")

## Comparing Network Formats

Let's compare how the same transaction looks across different networks:

In [ ]:
# Common transaction data
common_fields = {
    3: "000000",
    4: "000000005000",
    11: "111222",
    14: "2612",
    22: "051",
    41: "TERM0003",
    42: "COMPARISON1234 ",
    49: "840",
}

networks_and_pans = [
    (CardNetwork.VISA, "4111111111111111"),
    (CardNetwork.MASTERCARD, "5555555555554444"),
    (CardNetwork.AMEX, "378282246310005"),
    (CardNetwork.DISCOVER, "6011111111111117"),
]

print("Same Transaction - Different Networks:")
print("=" * 60)

for network, pan in networks_and_pans:
    msg = ISO8583Message(mti="0100", network=network, fields={0: "0100", 2: pan, **common_fields})
    raw = builder.build(msg)
    print(f"\n{network.name}:")
    print(f"  PAN: {pan}")
    print(f"  Message length: {len(raw)}")
    print(f"  Raw (first 50): {raw[:50]}")

## BIN Ranges and Card Detection

Different networks have different BIN (Bank Identification Number) ranges:

In [ ]:
def detect_network(pan: str) -> str:
    """Detect card network from PAN."""
    if pan.startswith("4"):
        return "VISA"
    elif pan.startswith(("51", "52", "53", "54", "55")):
        return "Mastercard"
    elif pan.startswith(("2221", "2222", "2223", "2224", "2225", "2226", "2227", "2228", "2229")):
        return "Mastercard (2-series)"
    elif pan.startswith(("23", "24", "25", "26", "27")):
        return "Mastercard (2-series)"
    elif pan.startswith(("34", "37")):
        return "American Express"
    elif pan.startswith("6011") or pan.startswith("65") or pan.startswith("644"):
        return "Discover"
    elif pan.startswith("62"):
        return "UnionPay"
    elif pan.startswith(("30", "36", "38", "39")):
        return "Diners Club"
    elif pan.startswith("35"):
        return "JCB"
    else:
        return "Unknown"


test_pans = [
    "4111111111111111",
    "5555555555554444",
    "2223000048400011",
    "378282246310005",
    "6011111111111117",
    "6200000000000005",
    "3566002020360505",
]

print("Card Network Detection:")
print("-" * 50)
for pan in test_pans:
    network = detect_network(pan)
    print(f"{pan[:6]}... -> {network}")

## Merchant Category Codes (MCC)

Field 18 contains the MCC, which varies by network but follows ISO 18245:

In [ ]:
mcc_examples = {
    "5411": "Grocery Stores, Supermarkets",
    "5541": "Service Stations (with or without ancillary services)",
    "5812": "Eating Places and Restaurants",
    "5814": "Fast Food Restaurants",
    "5912": "Drug Stores and Pharmacies",
    "5942": "Book Stores",
    "5999": "Miscellaneous and Specialty Retail Stores",
    "6011": "Financial Institutions - Automated Cash Disbursements",
    "7011": "Lodging - Hotels, Motels, Resorts",
    "7832": "Motion Picture Theaters",
    "8011": "Doctors and Physicians",
}

print("Common Merchant Category Codes (Field 18):")
print("-" * 60)
for mcc, description in mcc_examples.items():
    print(f"MCC {mcc}: {description}")

## POS Entry Mode (Field 22)

Field 22 indicates how the card data was captured:

In [ ]:
pos_entry_modes = {
    "00": "Unknown",
    "01": "Manual entry (key-entered)",
    "02": "Magnetic stripe read",
    "05": "Chip card read (ICC)",
    "07": "Contactless chip",
    "09": "E-commerce (card not present)",
    "10": "Credential on file",
    "80": "Chip fallback to magnetic stripe",
    "81": "E-commerce, secure (3DS)",
    "91": "Contactless magnetic stripe",
}

pin_capability = {
    "0": "Unknown",
    "1": "Can accept PIN",
    "2": "Cannot accept PIN",
    "8": "PIN pad inoperative",
}

print("POS Entry Mode (Field 22):")
print("-" * 50)
print("Format: XXY where XX=entry mode, Y=PIN capability")
print("\nEntry Modes:")
for code, desc in pos_entry_modes.items():
    print(f"  {code}: {desc}")
print("\nPIN Capability (3rd digit):")
for code, desc in pin_capability.items():
    print(f"  {code}: {desc}")

## Response Codes by Network

While response codes are somewhat standardized, there are network-specific variations:

In [ ]:
# Common response codes (most networks)
response_codes = {
    "00": "Approved",
    "01": "Refer to card issuer",
    "02": "Refer to card issuer, special condition",
    "03": "Invalid merchant",
    "04": "Pick up card",
    "05": "Do not honor",
    "06": "Error",
    "07": "Pick up card, special condition",
    "08": "Honor with identification",
    "10": "Partial approval",
    "12": "Invalid transaction",
    "13": "Invalid amount",
    "14": "Invalid card number",
    "15": "No such issuer",
    "30": "Format error",
    "41": "Lost card, pick up",
    "43": "Stolen card, pick up",
    "51": "Insufficient funds",
    "54": "Expired card",
    "55": "Incorrect PIN",
    "57": "Transaction not permitted to cardholder",
    "58": "Transaction not permitted to terminal",
    "61": "Exceeds withdrawal amount limit",
    "62": "Restricted card",
    "63": "Security violation",
    "65": "Exceeds withdrawal frequency limit",
    "75": "Allowable number of PIN tries exceeded",
    "76": "Invalid/nonexistent account",
    "78": "No account of type requested",
    "80": "Visa transactions: credit issuer unavailable",
    "81": "PIN cryptographic error",
    "82": "Negative CAM, dCVV, iCVV, CVV results",
    "85": "No reason to decline (address verification)",
    "91": "Issuer or switch unavailable",
    "92": "Unable to route transaction",
    "94": "Duplicate transmission",
    "96": "System malfunction",
}

print("Response Codes (Field 39):")
print("-" * 50)
for code, desc in list(response_codes.items())[:15]:
    print(f"  {code}: {desc}")
print("  ...")
print(f"  (Total: {len(response_codes)} codes)")

## Next Steps

- **[05_emv_data.ipynb](05_emv_data.ipynb)** - Working with EMV/chip card data in Field 55